# CP-SAT: Decimal numbers

In [ ]:
from ortools.sat.python import cp_model

## Division with decimal numbers

CP-SAT is an integer programming solver. Thus, it only stores integer variables. 

But when you divide variables, you sometimes need more precision than an integer. 

Simply multiply the variable by a big number to get a rounded down approximation. 

In [ ]:
FLOAT_APPROX_PRECISION = 100

In [ ]:
model = cp_model.CpModel()

numerator = model.NewIntVar(0, 100, "numerator")
denominator = model.NewIntVar(1, 100, "denominator")
result = model.NewIntVar(0, 100, "result")

# We want to compute 10/30
model.Add(numerator == 10)
model.Add(denominator == 30)
# Add a divison equality and multiply numerator by 100
model.AddDivisionEquality(result, numerator * FLOAT_APPROX_PRECISION, denominator)

# Solve
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Divide the result to get a rounded down solution
print(f"Solution is approximately: {solver.Value(result) / FLOAT_APPROX_PRECISION}")


Solution is approximately: 0.33


## Constraints with decimal numbers

Let's say you want the result variable to be more than half of the variable x. This will fail.

In [ ]:
model = cp_model.CpModel()

x = model.NewIntVar(0, 100, "x")
result = model.NewIntVar(0, 100, "result")

# Assume x == 30
model.Add(numerator == 30)
# This will fail
model.Add(result >= 0.5 * x)

TypeError: Not an integer: -0.5

But you can rewrite this inequality, to say that the variable result should be at least twice as big as the variable x.

In this casee, the constraint is perfectly valid.

In [ ]:
model.Add(2 * result >= x)

More generally, a quick and dirty way to handle constraints with decimal numbers is to multiply both sides by a big number and round everything. 

In [ ]:
# This will work well enough in most cases
model.Add(FLOAT_APPROX_PRECISION*result >= round(FLOAT_APPROX_PRECISION * 0.5) * x)